### **Garbage detection project**

This Colab Notebook shows a simple way on how to use the Tensorflow API to 
detect custom objects from images.

In my case I tried to detect garbage within images that contain glass, paper and alu objects. I've created necessary images myself. The images have been taken within my private household to create a realistic dataset for garbage detection in private households.

Within this project I used my own Google Drive folders to synchronize between difference devices. This happens by mounting the Google Drive folders to this Colab Notebook.

### 0.) **Setup the environment**



1.   Load and install necessary packages (tf version 1.15.0 is needed here).
2.   Mount Colab Notebook to Google Drive folder.



In [ ]:
# How much hours are left for the current session?

import time, psutil
Start = time.time()- psutil.boot_time()
Left= 12*3600 - Start
print('Time remaining for this session is: ', Left/3600)

Time remaining for this session is:  11.981702216333813


In [20]:
# IMPORTS
from __future__ import division, print_function, absolute_import

import pandas as pd
import numpy as np
import csv

import re
import os
import io
import glob
import shutil
import urllib.request
import tarfile
import xml.etree.ElementTree as ET

import cv2 

from PIL import Image
from collections import namedtuple, OrderedDict

from google.colab import files

In [21]:
# Initial module installation
!pip install tensorflow==1.15.0
!pip install tf_slim
!pip install -qq Cython contextlib2 pillow lxml matplotlib pycocotools
!pip install lvis

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

     |████████████████████████████████| 412.3MB 42kB/s 
     |████████████████████████████████| 3.8MB 51.8MB/s 
     |████████████████████████████████| 512kB 49.5MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=b9bbea392037bb7d376fc4373397916fd7faf5dc3e45953cda0b4421d82ab469
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tens

In [22]:
import tensorflow as tf
# Version 1.15.0 is required
print(tf.__version__)

1.15.0


In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
%cd '/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector'

/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector


### 1. ) ***Preprocess data***



1.   Create a folder for test and train data each.
2.   Move a part of the image annotations from the dataset to the test folder and the rest to the train folder (I used a 1/3 - 2/3 split respectively).
3.   Create one CSV file for the train and test annotations respectively (from XML annotation files) plus a label map file.
4.   Clone the tensorflow model repository.
5.   Setup PATH variable to the `slim` directory of `tensorflow/models`
6.   Run protobuf compilers.
7.   Create tf record files (.record) for train and test data.


In [ ]:
%cd data

/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/data


In [ ]:
!mkdir test_labels train_labels

In [ ]:
## Move to ./data dir

# Move the first x annotations to test
!ls annotations/* | sort -R | head -37 | xargs -I{} mv {} test_labels/

# Move the rest (401) to train
!ls annotations/* | xargs -I{} mv {} train_labels/

In [ ]:
## Move to ./data dir

#adjusted from: https://github.com/datitran/raccoon_dataset
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)

Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.


 **Setting up models folder and protoc compiler**

In [ ]:
# Get tensorflow models repo
!git clone https://github.com/tensorflow/models.git

In [ ]:
# New data
%cd ../models/research/

[Errno 2] No such file or directory: '../models/research/'
/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector


In [ ]:
# No new data
%cd ./models/research/

[Errno 2] No such file or directory: './models/research/'
/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/models/research


In [ ]:
# THIS NEEDS TO GET SPECIFIED EVERY TIME
# exports PYTHONPATH environment var with research and slim paths
os.environ['PYTHONPATH'] += ':./:./slim/'

In [ ]:
# Compile proto buffers
!protoc ./object_detection/protos/*.proto --python_out=.

In [ ]:
# Check if modelbuilder is working properly
!python3 ./object_detection/builders/model_builder_test.py

**End of setup**

In [ ]:
## Move to ./data dir

#adjusted from: https://github.com/datitran/raccoon_dataset
from object_detection.utils import dataset_util

#change this to the base directory where your data/ is 
data_base_url = '/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/data/'

#location of images
image_dir = data_base_url +'images/'

def class_text_to_int(row_label):
  if row_label == 'Papiertonne':
    return 1
  if row_label == 'Gelber Sack':
    return 2
  if row_label == 'Glascontainer':
    return 3
  else:
    None

def split(df, group):
  data = namedtuple('data', ['filename', 'object'])
  gb = df.groupby(group)
  return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
  with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
    encoded_jpg = fid.read()
  
  encoded_jpg_io = io.BytesIO(encoded_jpg)
  image = Image.open(encoded_jpg_io)
  width, height = image.size
  filename = group.filename.encode('utf8')
  image_format = b'jpg'
  xmins = []
  xmaxs = []
  ymins = []
  ymaxs = []
  classes_text = []
  classes = []

  for index, row in group.object.iterrows():
    xmins.append(row['xmin'] / width)
    xmaxs.append(row['xmax'] / width)
    ymins.append(row['ymin'] / height)
    ymaxs.append(row['ymax'] / height)
    classes_text.append(row['class'].encode('utf8'))
    classes.append(class_text_to_int(row['class']))

  tf_example = tf.train.Example(features=tf.train.Features(feature={
    'image/height': dataset_util.int64_feature(height),
    'image/width': dataset_util.int64_feature(width),
    'image/filename': dataset_util.bytes_feature(filename),
    'image/source_id': dataset_util.bytes_feature(filename),
    'image/encoded': dataset_util.bytes_feature(encoded_jpg),
    'image/format': dataset_util.bytes_feature(image_format),
    'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
    'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
    'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
    'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
    'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
    'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
  return tf_example

#creates tfrecord for both csv's
for csv in ['train_labels', 'test_labels']:
  writer = tf.io.TFRecordWriter(data_base_url + csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(data_base_url + csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), data_base_url + csv + '.record')
  print('Successfully created the TFRecords: {}'.format(data_base_url +csv + '.record'))

Successfully created the TFRecords: /gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/data/train_labels.record
Successfully created the TFRecords: /gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/data/test_labels.record


### 2.) **Configure model and train it**



1.   Specify model to use (`selected_model`)
2.   Load predefined model from the tensorflow repository and store it.
3.   Adjust the config file for that model to your needs (I compared Faster RCNN with MobileNet).
4.   Create a tunnel with `ngrok` to run the tensorboard from within this notebook.
5.   Start the training.



In [ ]:
# Some models to train on
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
    },
}

# Select a model from `MODELS_CONFIG`.
# I chose ssd_mobilenet_v2 for this project, you could choose any
selected_model = 'ssd_mobilenet_v2'

In [ ]:
#the distination folder where the model will be saved
#change this if you have a different working dir
DEST_DIR = '/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/models/research/pretrained_model'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

#selecting the model
MODEL_FILE = MODEL + '.tar.gz'

#creating the downlaod link for the model selected
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

#checks if the model has already been downloaded, download it otherwise
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#unzipping the model and extracting its content
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# creating an output file to save the model while training
os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

In [ ]:
#path to the sample config file
#!cat object_detection/samples/configs/ssd_mobilenet_v2_coco.config
!cat object_detection/samples/configs/faster_rcnn_inception_v2_coco.config

In [ ]:
## Config for faster rcnn
%%writefile object_detection/samples/configs/faster_rcnn_inception_v2_coco.config

# Faster R-CNN with Inception v2, configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.


model {
  faster_rcnn {
    num_classes: 3
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 600
        max_dimension: 1024
      }
    }
    feature_extractor {
      type: 'faster_rcnn_inception_v2'
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        scales: [0.25, 0.5, 1.0, 2.0]
        aspect_ratios: [0.5, 1.0, 2.0]
        height_stride: 16
        width_stride: 16
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.01
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.7
    first_stage_max_proposals: 300
    first_stage_localization_loss_weight: 2.0
    first_stage_objectness_loss_weight: 1.0
    initial_crop_size: 14
    maxpool_kernel_size: 2
    maxpool_stride: 2
    second_stage_box_predictor {
      mask_rcnn_box_predictor {
        use_dropout: false
        dropout_keep_probability: 1.0
        fc_hyperparams {
          op: FC
          regularizer {
            l2_regularizer {
              weight: 0.0
            }
          }
          initializer {
            variance_scaling_initializer {
              factor: 1.0
              uniform: true
              mode: FAN_AVG
            }
          }
        }
      }
    }
    second_stage_post_processing {
      batch_non_max_suppression {
        score_threshold: 0.0
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 300
      }
      score_converter: SOFTMAX
    }
    second_stage_localization_loss_weight: 2.0
    second_stage_classification_loss_weight: 1.0
  }
}

train_config: {
  batch_size: 1
  optimizer {
    momentum_optimizer: {
      learning_rate: {
        manual_step_learning_rate {
          initial_learning_rate: 0.0002
          schedule {
            step: 900000
            learning_rate: .00002
          }
          schedule {
            step: 1200000
            learning_rate: .000002
          }
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
  gradient_clipping_by_norm: 10.0
  fine_tune_checkpoint: "/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/models/research/pretrained_model/model.ckpt"
  from_detection_checkpoint: true
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the COCO dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 200000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    input_path: "/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/data/train_labels.record"
  }
  label_map_path: "/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/data/label_map.pbtxt"
}

eval_config: {
  num_examples: 8000
  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  max_evals: 10
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/data/test_labels.record"
  }
  label_map_path: "/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/data/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

Overwriting object_detection/samples/configs/faster_rcnn_inception_v2_coco.config


In [ ]:
#path to the config file
%%writefile object_detection/samples/configs/ssd_mobilenet_v2_coco.config

# paste the content of the config file in the same cell here.
# SSD with Mobilenet v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 3
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.3333
      }
    }
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: false
        #use_dropout: true
        dropout_keep_probability: 0.8
        kernel_size: 1
        box_code_size: 4
        apply_sigmoid_to_scores: false
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
              weight: 0.00004
            }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
          batch_norm {
            train: true,
            scale: true,
            center: true,
            decay: 0.9997,
            epsilon: 0.001,
          }
        }
      }
    }
    feature_extractor {
      type: 'ssd_mobilenet_v2'
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            weight: 0.00004
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.9997,
          epsilon: 0.001,
        }
      }
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000
        iou_threshold: 0.99
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 3
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        max_detections_per_class: 8
        max_total_detections: 8
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  batch_size: 32
  optimizer {
    rms_prop_optimizer: {
      learning_rate: {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.01
          decay_steps: 800720
          decay_factor: 0.95
        }
      }
      momentum_optimizer_value: 0.9
      decay: 0.9
      epsilon: 1.0
    }
  }
  fine_tune_checkpoint: "/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/models/research/pretrained_model/model.ckpt"
  fine_tune_checkpoint_type:  "detection"
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 200000
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
  data_augmentation_options {
    random_adjust_contrast {
    }
  }
  data_augmentation_options {
    random_rgb_to_gray {
    }
  }
  data_augmentation_options {
    random_vertical_flip {
    }
  }
  data_augmentation_options {
    random_rotation90 {
    }
  }
  data_augmentation_options {
    random_patch_gaussian {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    input_path: "/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/data/train_labels.record"
  }
  label_map_path: "/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/data/label_map.pbtxt"
}

eval_config: {
  num_examples: 37
  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  #max_evals: 10
  num_visualizations: 20
}

eval_input_reader: {
  tf_record_input_reader {
    input_path: "/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/data/test_labels.record"
  }
  label_map_path: "/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/data/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

Overwriting object_detection/samples/configs/ssd_mobilenet_v2_coco.config


In [ ]:
# Getting tensorboard ready for colab
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-11-12 18:51:12--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.200.34.1, 54.85.41.146, 34.194.108.77, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.200.34.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.14’

ngrok-stable-linux- 100%[===================>]  13.13M  35.5MB/s    in 0.4s    

2020-11-12 18:51:12 (35.5 MB/s) - ‘ngrok-stable-linux-amd64.zip.14’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
#specify where the log files are stored and we configure a link to view Tensorboard

#the logs that are created while training 
LOG_DIR = "training/"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
#The link to tensorboard.
#works after the training starts.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://e1ab9e95691c.ngrok.io


In [ ]:
# Start the training... !!!
# MobileNet Config: 
#--pipeline_config_path=/gdrive/My\ Drive/Ausbildung/Studium/FOM/\3\.\ Semester/Anwendungsfelder\ Business\ Analytics/Colab\ Notebooks/garbage_detector/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config \
#--pipeline_config_path=/gdrive/My\ Drive/Ausbildung/Studium/FOM/\3\.\ Semester/Anwendungsfelder\ Business\ Analytics/Colab\ Notebooks/garbage_detector/models/research/object_detection/samples/configs/faster_rcnn_inception_v2_coco.config \
!python3 object_detection/model_main.py \
    --pipeline_config_path=/gdrive/My\ Drive/Ausbildung/Studium/FOM/\3\.\ Semester/Anwendungsfelder\ Business\ Analytics/Colab\ Notebooks/garbage_detector/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config \
    --model_dir=training/

stopp bei 31700

### 3.) **Export model and start visual inference**



1.   Specify the location where the exported inference graph should be stored.
2.   Export inference graph with Tensorflow API.



In [ ]:
%cd ./models/research/

[Errno 2] No such file or directory: './models/research/'
/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/models/research


In [ ]:
#dir where the model will be saved
output_directory = './fine_tuned_model_mobilenet_my_data_v2_300x300_32000s'

lst = os.listdir('training')
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join('training', last_model)

In [ ]:
# os.environ['PYTHONPATH'] += ':./:./slim/' has to be set
# MobileNet config
#/ssd_mobilenet_v2_coco.config \
#/faster_rcnn_inception_v2_coco.config \

!python ./object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=/gdrive/My\ Drive/Ausbildung/Studium/FOM/\3\.\ Semester/Anwendungsfelder\ Business\ Analytics/Colab\ Notebooks/garbage_detector/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}


Instructions for updating:
Please use `layer.__call__` method instead.
W1112 20:05:04.113483 139630500751232 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tf_slim/layers/layers.py:1089: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.


### 4.) **Start local inference**

Code snippet that can be used locally togehter with e.g. a webcam to run inference.



In [ ]:
# Code to run local inference with webcam

# This code need to be executed next to the models/research/object_detection/utils folder for the utils import.
# Also the protobuf compiler needs to get installed locally and run against the protobuf files from the tensorflow repo.
import numpy as np
import os
import tensorflow as tf
import cv2
from utils import label_map_util
from utils import visualization_utils as vis_util

# path to the frozen graph:
PATH_TO_FROZEN_GRAPH = '/Users/aj/Google Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/models/research/pretrained_model/frozen_inference_graph.pb'

# path to the label map
PATH_TO_LABEL_MAP = '/Users/aj/Google Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/data/label_map.pbtxt'

# number of classes 
NUM_CLASSES = 1

cap = cv2.VideoCapture(0)

#reads the frozen graph
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABEL_MAP)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Detection
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        while True:
            # Read frame from camera
            ret, image_np = cap.read()
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Extract image tensor
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Extract detection boxes
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Extract detection scores
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            # Extract detection classes
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            # Extract number of detections
            num_detections = detection_graph.get_tensor_by_name(
                'num_detections:0')
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=3,
                )
        # Display output
            cv2.imshow('Gun Detection', cv2.resize(image_np, (1200, 800)))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break

### 5.) **Show attributes of the inference graphs**

1.   Define all models that will get inspected.
2.   Use `graph_metrics.py` from the tensorflow API to extract metrics from the trained inference graphs.
[Link to the specific tensorflow blob](https://raw.githubusercontent.com/tensorflow/tensorflow/fe454464681b036ff7fed3e42c6bb541fa52dd7c/tensorflow/python/tools/graph_metrics.py)


In [23]:
RESEARCH_PATH = '/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/models/research/'

inference_graphs = {
  'Mobilenet own data v1': RESEARCH_PATH + 'fine_tuned_model_mobilenet_my_data_v1_300x300_31752s',
  'Mobilenet own data v2': RESEARCH_PATH + 'fine_tuned_model_mobilenet_my_data_v2_300x300_32000s',
  'Mobilenet trashnet data v1': RESEARCH_PATH + 'fine_tuned_model_mobilenet_v1',
  'Mobilenet trashnet data v2': RESEARCH_PATH + 'fine_tuned_model_mobilenet_v2',
  'Faster RCNN trashnet data v1': RESEARCH_PATH + 'fine_tuned_model_rcnn_v1'   
}

In [ ]:
for model, path in inference_graphs.items():
  print(f'Extracting attributes of inference graph for model {model}... \n')

  frozen_inference_graph = path + '/frozen_inference_graph.pb'
  #--graph frozen_inference_graph \

  !python '/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/tools/graph_metrics.py' \
    --graph '/gdrive/My Drive/Ausbildung/Studium/FOM/3. Semester/Anwendungsfelder Business Analytics/Colab Notebooks/garbage_detector/models/research/fine_tuned_model_mobilenet_my_data_v2_300x300_32000s/frozen_inference_graph.pb' \
    --statistics=weight_parameters,flops

  print('\n\n\n')